In [3]:
# Dependencies
from flask import Flask, render_template, redirect, Markup, jsonify
from flask_pymongo import PyMongo
from bson.json_util import dumps
import pymongo
from flask_cors import CORS
import pandas as pd
import json

In [4]:
app = Flask(__name__)

CORS(app, support_credentials = True)

# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/election2020"
mongo = PyMongo(app)

In [23]:
# create route that rendrs index.html template
# @app.route("/")
def index():
    
    # Create dictionary to for topojson state ids
    state_ids = {"Alabama":"01",
                 "Alaska":"02",
                 "Arizona":"04",
                 "Arkansas":"05",
                 "California":"06",
                 "Colorado":"08",
                 "Connecticut": "09",
                 "Delaware":"10",
                 "District of Columbia":"11",
                 "Florida":"12",
                 "Georgia":"13",
                 "Hawaii":"15",
                 "Idaho": "16",
                 "Illinois": "17",
                 "Indiana": "18",
                 "Iowa": "19",
                 "Kansas":"20",
                 "Kentucky":"21",
                 "Louisiana":"22",
                 "Maine":"23",
                 "Maryland":"24" ,
                 "Massachusetts":"25",
                 "Michigan":"26",
                 "Minnesota":"27",
                 "Mississippi":"28",
                 "Missouri": "29",
                 "Montana":"30",
                 "Nebraska":"31",
                 "Nevada": "32",
                 "New Hampshire":"33",
                 "New Jersey": "34",
                 "New Mexico": "35",
                "New York":"36",
                "North Carolina": "37",
                "North Dakota": "38",
                "Ohio":"39",
                "Oklahoma": "40",
                "Oregon": "41",
                "Pennsylvania": "42",
                "Rhode Island":"44",
                "South Carolina":"45",
                "South Dakota": "46",
                "Tennessee":"47",
                "Texas": "48",
                 "Utah":"49",
                 "Vermont":"50",
                 "Virginia":"51",
                 "Washington":"53",
                 "West Virginia": "54",
                 "Wisconsin":"55",
                 "Wyoming": "56"}

    # Pull data from Mongo
    candidate = mongo.db.pCandidate.find()
    state = mongo.db.pState.find()
    county = mongo.db.pCounty.find()
    county_ids = mongo.db.countyIds.find()

    all_data = [candidate, state, county, county_ids]

    return_list = []

    for dataset in all_data:

        list_cur = list(dataset)
        json_data = dumps(list_cur)
        return_list.append(json_data)
    
    # Pull only candidate data
    candidate_data = return_list[0]
    
    # Convert string of json to a json file
    candidate_json = json.loads(candidate_data)
    
    # Convert json file to data frame
    df = pd.DataFrame.from_records(candidate_json)
    
    # Filter data by our two candidates (Biden and Trump)
    candidate_df = df.loc[(df["candidate"] == "Joe Biden") | (df["candidate"] == "Donald Trump"),:]
    
    # Group data by states
    condensed_df = candidate_df.groupby(["state","candidate"]).sum()
    
    # Calculate Democrat win percentage
    i = 0         # 0 means Trump, 1 means Biden 
    count = 0
    finalJson = {}
    percentDem= []
    x = 1 
       
    for index, row in condensed_df.iterrows():
        if i == 0:
            count = count + row["total_votes"]
            i = i + 1
        else:
            count = count + row["total_votes"]
            percent = row["total_votes"] / count
            state = row.name[0]
            state_id = state_ids.get(state)
            
            state_info = {state_id : percent}

            percentDem.append(state_info)

            i = 0
            count = 0
            x= x + 1
    
    countyIds_data = return_list[3]
    countyIds_json = json.loads(countyIds_data)
    
    finalJson.update({"percentDem": percentDem})
    finalJson.update({"countyIDs": countyIds_json})
    
    return finalJson



#if __name__ == "__main__":
    #app.run(debug=True)

In [24]:
d = index()
d

{'percentDem': [{'01': 0.37089316645247805},
  {'02': 0.44685796846462394},
  {'04': 0.5015683167912931},
  {'05': 0.3578756672201685},
  {'06': 0.649096897735854},
  {'08': 0.5693831949230015},
  {'09': 0.6017246380927086},
  {'10': 0.5962674416498447},
  {'11': 0.9446695384761944},
  {'12': 0.48305245337858443},
  {'13': 0.5011932966290864},
  {'15': 0.6503266464651488},
  {'16': 0.3412285707492213},
  {'17': 0.6009757698055935},
  {'18': 0.4180181519856194},
  {'19': 0.4581673691536295},
  {'20': 0.42427259450974314},
  {'21': 0.36799897099737033},
  {'22': 0.4053555954370895},
  {'23': 0.5446460653970535},
  {'24': 0.6702904704709234},
  {'25': 0.6711554954014815},
  {'26': 0.5141355934440947},
  {'27': 0.5363951364856667},
  {'28': 0.4161501417261114},
  {'29': 0.4216417935559855},
  {'30': 0.41602819907951893},
  {'31': 0.40215947753398273},
  {'32': 0.512231173400438},
  {'33': 0.5374834772159505},
  {'34': 0.5807128358679484},
  {'35': 0.5551849015171975},
  {'36': 0.6173605254

In [ ]:
b = pd.DataFrame.from_records(d)

c = b.loc[(b["candidate"] == "Joe Biden") | (b["candidate"] == "Donald Trump"),:]

i = 0
count = 0
percentDem= []

for index, row in c.iterrows():
    if i == 0:
        count = count + row["total_votes"]
        i = i + 1
    else:
        count = count + row["total_votes"]
        
        percent = row["total_votes"] / count
        
        percentDem.append(percent)
        
        i = 0
        count = 0

percentDem

In [41]:
    candidate = mongo.db.pCandidate.find()
    state = mongo.db.pState.find()
    county = mongo.db.pCounty.find()

    all_data = [candidate, state, county]

    return_list = []

    for dataset in all_data:

        list_cur = list(dataset)
        json_data = dumps(list_cur)
        return_list.append(json_data)
    
    # Pull only candidate data
    candidate_data = return_list[0]
    
    # Convert string of json to a json file
    candidate_json = json.loads(candidate_data)
    
    # Convert json file to data frame
    df = pd.DataFrame.from_records(candidate_json)
    
    # Filter data by our two candidates (Biden and Trump)
    candidate_df = df.loc[(df["candidate"] == "Joe Biden") | (df["candidate"] == "Donald Trump"),:]
    
    # Group data by states
    condensed_df = candidate_df.groupby(["state","candidate"]).sum()
    
    condensed_df



total_votes   won
state         candidate                      
Alabama       Donald Trump      1441168  54.0
              Joe Biden          849648  13.0
Alaska        Donald Trump       189892  20.0
              Joe Biden          153405  17.0
Arizona       Donald Trump      1661686  10.0
...                                 ...   ...
West Virginia Joe Biden          235984   0.0
Wisconsin     Donald Trump      1610065  58.0
              Joe Biden         1630673  14.0
Wyoming       Donald Trump       193559  21.0
              Joe Biden           73491   2.0

[102 rows x 2 columns]

In [45]:
    for index, row in condensed_df.iterrows():
        print(row.name[0])

Alabama
Alabama
Alaska
Alaska
Arizona
Arizona
Arkansas
Arkansas
California
California
Colorado
Colorado
Connecticut
Connecticut
Delaware
Delaware
District of Columbia
District of Columbia
Florida
Florida
Georgia
Georgia
Hawaii
Hawaii
Idaho
Idaho
Illinois
Illinois
Indiana
Indiana
Iowa
Iowa
Kansas
Kansas
Kentucky
Kentucky
Louisiana
Louisiana
Maine
Maine
Maryland
Maryland
Massachusetts
Massachusetts
Michigan
Michigan
Minnesota
Minnesota
Mississippi
Mississippi
Missouri
Missouri
Montana
Montana
Nebraska
Nebraska
Nevada
Nevada
New Hampshire
New Hampshire
New Jersey
New Jersey
New Mexico
New Mexico
New York
New York
North Carolina
North Carolina
North Dakota
North Dakota
Ohio
Ohio
Oklahoma
Oklahoma
Oregon
Oregon
Pennsylvania
Pennsylvania
Rhode Island
Rhode Island
South Carolina
South Carolina
South Dakota
South Dakota
Tennessee
Tennessee
Texas
Texas
Utah
Utah
Vermont
Vermont
Virginia
Virginia
Washington
Washington
West Virginia
West Virginia
Wisconsin
Wisconsin
Wyoming
Wyoming
